In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [4]:
credentials_location = '/home/ojekky/.gc/my-creds.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/home/ojekky/data-engineering-zoomcamp/05-batch/code/lib/gcs-connector-hadoop2-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [5]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

25/04/11 21:10:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [7]:
!wget https://s3.amazonaws.com/capitalbikeshare-data/201801-capitalbikeshare-tripdata.zip

--2025-04-11 21:14:23--  https://s3.amazonaws.com/capitalbikeshare-data/201801-capitalbikeshare-tripdata.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.227.56, 16.182.38.40, 52.216.216.48, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.227.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3870111 (3.7M) [application/zip]
Saving to: ‘201801-capitalbikeshare-tripdata.zip’

201801-capitalbikes 100%[===================>]   3.69M  6.54MB/s    in 0.6s    

2025-04-11 21:14:24 (6.54 MB/s) - ‘201801-capitalbikeshare-tripdata.zip’ saved [3870111/3870111]



In [8]:
!wget https://s3.amazonaws.com/capitalbikeshare-data/202412-capitalbikeshare-tripdata.zip

--2025-04-11 21:14:51--  https://s3.amazonaws.com/capitalbikeshare-data/202412-capitalbikeshare-tripdata.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.184.20, 52.216.249.166, 52.216.113.237, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.184.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14865654 (14M) [application/zip]
Saving to: ‘202412-capitalbikeshare-tripdata.zip’

202412-capitalbikes 100%[===================>]  14.18M  15.5MB/s    in 0.9s    

2025-04-11 21:14:52 (15.5 MB/s) - ‘202412-capitalbikeshare-tripdata.zip’ saved [14865654/14865654]



In [12]:
door = spark.read \
    .option("header", "true") \
    .csv('201801_capitalbikeshare_tripdata.csv')

In [13]:
dor = !head -n 10 201801_capitalbikeshare_tripdata.csv

In [14]:
dur = !head -n 10 202412-capitalbikeshare-tripdata.csv

In [17]:
dor

['"Duration","Start date","End date","Start station number","Start station","End station number","End station","Bike number","Member type"',
 '552,"2018-01-01 00:05:06","2018-01-01 00:14:18",31104,"Adams Mill & Columbia Rd NW",31400,"Georgia & New Hampshire Ave NW","W00886","Member"',
 '1282,"2018-01-01 00:14:30","2018-01-01 00:35:53",31321,"15th St & Constitution Ave NW",31321,"15th St & Constitution Ave NW","W01435","Casual"',
 '1265,"2018-01-01 00:14:53","2018-01-01 00:35:58",31321,"15th St & Constitution Ave NW",31321,"15th St & Constitution Ave NW","W21242","Casual"',
 '578,"2018-01-01 00:15:31","2018-01-01 00:25:09",31406,"14th & Upshur St NW",31103,"16th & Harvard St NW","W21322","Casual"',
 '372,"2018-01-01 00:18:02","2018-01-01 00:24:15",31618,"4th & East Capitol St NE",31619,"Lincoln Park / 13th & East Capitol St NE ","W00119","Member"',
 '369,"2018-01-01 00:18:07","2018-01-01 00:24:17",31618,"4th & East Capitol St NE",31619,"Lincoln Park / 13th & East Capitol St NE ","W21076

In [18]:
dur

['"ride_id","rideable_type","started_at","ended_at","start_station_name","start_station_id","end_station_name","end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual"',
 '"30BF1BA07E61B962","electric_bike","2024-12-19 15:00:37.692","2024-12-19 15:06:27.690",,,,,38.93,-77.04,38.92,-77.03,"member"',
 '"9990ACCC341A77E1","electric_bike","2024-12-19 15:10:30.222","2024-12-19 15:15:33.420",,,,,38.89,-77.09,38.89,-77.09,"member"',
 '"B676007C128ED93B","electric_bike","2024-12-19 15:41:54.507","2024-12-19 16:07:07.430",,,,,38.92,-77.0,38.98,-77.0,"member"',
 '"75B0EED9CAB23C5B","electric_bike","2024-12-18 20:39:43.412","2024-12-18 20:48:04.109",,,,,38.86,-77.11,38.86,-77.12,"member"',
 '"2774271B0FC6690F","electric_bike","2024-12-18 19:37:46.065","2024-12-18 19:40:37.214",,,,,38.86,-77.11,38.86,-77.11,"member"',
 '"7CFB12BCF8F83D9E","electric_bike","2024-12-15 09:09:08.302","2024-12-15 09:16:44.017",,,,,38.93,-77.03,38.91,-77.03,"member"',
 '"3718BC0A1213FBCB","electric_b

In [21]:
df_recent = spark.read.csv(
    [
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/202[1-5]/*",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202005-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202006-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202007-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202008-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202009-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202010-capitalbikeshare-tripdata.csv"
    ], 
    header=True, 
    inferSchema=True
)

In [22]:
df_old = spark.read.csv(
    [
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/201[0-9]/*",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202001-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202002-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202003-capitalbikeshare-tripdata.csv"
    ], 
    header=True, 
    inferSchema=True
)

In [23]:
import pandas as pd

In [24]:
df_env = df_recent.head(100)

In [25]:
spark.createDataFrame(df_env).schema

StructType([StructField('ride_id', StringType(), True), StructField('rideable_type', StringType(), True), StructField('started_at', StringType(), True), StructField('ended_at', StringType(), True), StructField('start_station_name', StringType(), True), StructField('start_station_id', StringType(), True), StructField('end_station_name', StringType(), True), StructField('end_station_id', StringType(), True), StructField('start_lat', StringType(), True), StructField('start_lng', StringType(), True), StructField('end_lat', StringType(), True), StructField('end_lng', StringType(), True), StructField('member_casual', StringType(), True)])

In [28]:
df_eno = df_old.head(100)

In [29]:
spark.createDataFrame(df_eno).schema

StructType([StructField('Duration', LongType(), True), StructField('Start date', TimestampType(), True), StructField('End date', TimestampType(), True), StructField('Start station number', LongType(), True), StructField('Start station', StringType(), True), StructField('End station number', LongType(), True), StructField('End station', StringType(), True), StructField('Bike number', StringType(), True), StructField('Member type', StringType(), True)])

In [26]:
from pyspark.sql import types

In [30]:
df_env_schema = types.StructType([
    types.StructField('ride_id', types.IntegerType(), True), 
    types.StructField('rideable_type', types.IntegerType(), True), 
    types.StructField('started_at', types.TimestampType(), True), 
    types.StructField('ended_at', types.TimestampType(), True), 
    types.StructField('start_station_name', types.StringType(), True), 
    types.StructField('start_station_id', types.IntegerType(), True), 
    types.StructField('end_station_name', types.StringType(), True), 
    types.StructField('end_station_id', types.IntegerType(), True), 
    types.StructField('start_lat', types.DoubleType(), True), 
    types.StructField('start_lng', types.DoubleType(), True), 
    types.StructField('end_lat', types.DoubleType(), True), 
    types.StructField('end_lng', types.DoubleType(), True), 
    types.StructField('member_casual', types.IntegerType(), True)
])

df_eno_schema = types.StructType([
    types.StructField('Duration', types.LongType(), True), 
    types.StructField('Start date', types.TimestampType(), True), 
    types.StructField('End date', types.TimestampType(), True), 
    types.StructField('Start station number', types.IntegerType(), True), 
    types.StructField('Start station', types.StringType(), True), 
    types.StructField('End station number', types.IntegerType(), True), 
    types.StructField('End station', types.StringType(), True), 
    types.StructField('Bike number', types.IntegerType(), True), 
    types.StructField('Member type', types.IntegerType(), True)
])

In [31]:
df_recent = spark.read.csv(
    [
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/202[1-5]/*",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202005-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202006-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202007-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202008-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202009-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202010-capitalbikeshare-tripdata.csv"
    ], 
    header = True, 
    schema = df_env_schema,
    escape='"',
    nullValue="null"
)

df_recent \
    .repartition(1) \
    .write.parquet("gs://global-rookery-448215-m8_bike_data_raw/lat_bike_paq")

df_recent.printSchema()
print(f"Number of partitions: {df_recent.rdd.getNumPartitions()}")
print(f"Total records: {df_recent.count()}")


root
 |-- ride_id: integer (nullable = true)
 |-- rideable_type: integer (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: integer (nullable = true)

Number of partitions: 38


Total records: 19412400


In [ ]:
df_old = spark.read.csv(
    [
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/201[0-9]/*",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202001-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202002-capitalbikeshare-tripdata.csv",
        "gs://global-rookery-448215-m8_bike_data_raw/capitalbikeshare/2020/202003-capitalbikeshare-tripdata.csv"
    ], 
    header = True, 
    schema = df_eno_schema,
    escape='"',
    nullValue="null"
)

df_old \
    .repartition(1) \
    .write.parquet("gs://global-rookery-448215-m8_bike_data_raw/old_bike_paq")

df_old.printSchema()
print(f"Number of partitions: {df_old.rdd.getNumPartitions()}")
print(f"Total records: {df_old.count()}")